In [ ]:
using LinearAlgebra, Random, Printf
using JuMP, COSMO, CSDP
using Plots, LaTeXStrings
using Statistics, Distributions
using Optim, LineSearches
using LBFGSB

---
# Random problem generator

In [ ]:
"""
@article{Lewandowski2009a,
    Author = {Daniel Lewandowski and Dorota Kurowicka and Harry Joe},
    Journal = {Journal of Multivariate Analysis},
    Number = {9},
    Pages = {1989 - 2001},
    Title = {Generating random correlation matrices based on vines and extended onion method},
    Volume = {100},
    Year = {2009}}
https://doi.org/10.1016/j.jmva.2009.04.008
"""
function onion(n; η = 1.0)
    S = Symmetric(ones(n, n))
    β = η + (n-2)/2
    S.data[1,2] = 2*rand(Beta(β,β)) - 1
    for k = 2:n-1
        β -= 0.5
        y = rand(Beta(k/2, β))
        u = normalize!(randn(k))
        w = sqrt(y)*u
        F = cholesky(Symmetric(S[1:k,1:k]))
        S.data[1:k,k+1] = F.L*w
    end
    return S
end

function randncm(n; method=:onion, seed=0, γ=0.0, p=0.5)
    rng = Random.seed!(seed)
        
    # Random target matrix
    if method == :onion
        U = onion(n)
    else
        U = 2*rand(n,n) .- 1; U = Symmetric(triu(U,1) + I)
    end

    # Random noise
    E = Symmetric(triu(randn(n,n),1))
    U.data .= (1-γ).*U .+ γ.*E
    U = Symmetric(triu(U,1) + I)
    
    # Random sparse H
    H = [rand()<p ? 1.0 : 0.0 for i=1:n, j=1:n]
    H = Symmetric(triu(H,1))
    
    Random.seed!(rng)
    
    return U, H
end

In [ ]:
function ncm0(U, H)
    # Initialize model with correlation matrix constraints
    m = Model(with_optimizer(CSDP.Optimizer))
    #m = Model(with_optimizer(COSMO.Optimizer))
    MOI.set(m, MOI.Silent(), true)
    @variable(m, X[1:n,1:n], Symmetric)
    @constraint(m, diagcon, diag(X) .== 1)
    @constraint(m, psdcon, X in PSDCone())
    R = H.*(X .- U)
    @objective(m, Min, 0.5*dot(R, R))
    optimize!(m)
    Xval = value.(X)
    y = dual.(diagcon)
    fvals = [0.5*norm(H.*(Xval .- U))^2]
    return Xval, y, fvals
end

In [ ]:
n = 6
U, H = randncm(n, γ=1.0)
display(H.*U)
X, y, fvals = ncm0(U, H);
display(H.*X)
display(y)
@show fvals[end];

---
# JuMP.jl

In [ ]:
function ncm(U, H; method=:PG, accelerate=true, τ=1.0, kmax=30, verbose=true)
    
    issymmetric(U) || error("U must be symmetric")
    issymmetric(H) || error("H must be symmetric")
    
    n = size(U, 1)
    H2 = Symmetric(H.*H)
    
    M = Symmetric(zeros(n,n))
    
    # Loss function and gradient
    f(X) = 0.5*norm(M.data .= H.*(X .- U))^2
    ∇f(X) = Symmetric(H2.*(X .- U))

    # Lipschitz constant of ∇f
    L = norm(H2)

    # Quadratic model of f at Y
    Q(X,Y,L,τ) = f(Y) + dot(∇f(Y), X - Y) + L/2τ*dot(X - Y, X - Y)

    # Initialize model with correlation matrix constraints
    m = Model(with_optimizer(CSDP.Optimizer))
    #m = Model(with_optimizer(COSMO.Optimizer))
    MOI.set(m, MOI.Silent(), true)
    @variable(m, X[1:n,1:n], Symmetric)
    @constraint(m, diagcon, diag(X) .== 1)
    @constraint(m, psdcon, X in PSDCone())

    #X0 = Symmetric(diagm(ones(n)))
    X0 = Symmetric(zeros(n,n))
    fvals = [f(X0)]
    
    y = zeros(n)
    Λ = Symmetric(zeros(n,n))
    Γ = copy(Λ)
    V = copy(Λ)
    ∇fY = copy(Λ)
    Y = copy(X0)
    Xold = copy(X0)
    Xnew = copy(X0)
    t = 1.0
    
    for k = 1:kmax
        
        # Solve quadratic subproblem
        @objective(m, MOI.FEASIBILITY_SENSE, 0)
        @objective(m, Min, Q(X,Y,L,τ))
        optimize!(m)
        Xnew.data .= value.(X)

        # Ensure that diag(Xnew).==1 exactly
        d = diag(Xnew)
        for i=1:n
            d[i] = 1/sqrt(Xnew[i,i])
        end
        for j=1:n
            for i=1:n
                Xnew.data[i,j] = d[i]*d[j]*Xnew.data[i,j]
            end
        end
                
        # Evaluate loss function
        fnew = f(Xnew)
        Qnew = Q(Xnew,Y,L,τ)
        push!(fvals, fnew)

        # Obtain dual multipliers
        y .= dual.(diagcon)
        copy!(Λ, dual.(psdcon))
        
        # Compute subgradient Γ ∈ ∂g(X; ε)
        Γ.data .= -Λ .- Diagonal(y)

        # Compute dual residual
        ∇fY.data .= H2.*(Y .- U)
        V.data .= ∇fY .+ (L/τ).*(Xnew .- Y) .+ Γ

        for i=1:n
            d[i] = 1 - Xnew[i,i]
        end
        Rp = norm(d)/(1 + √n)
        Rd = norm(M.data .= H2.*(Xnew .- U) .- Diagonal(y) .- Λ)
        
        ε = dot(Xnew, Λ)
        δ = norm(V)
        dist = norm(M.data .= Xnew .- Y)
        
        if verbose
            mod(k, 20)==1 &&
            @printf("%4s %10s %10s %10s %10s %10s %10s\n", 
                "k", "f(X)", "Rp", "Rd", "<X,Λ>", "||V||", "||X-Y||")
            @printf("%4d %10.2e %10.2e %10.2e %10.2e %10.2e %10.2e\n", 
                k, fvals[end], Rp, Rd, ε, δ, dist)
        end
        
        #=
        if method==:IR
            (τ*δ)^2 + 2τ*ε*L ≤ (1-τ)*(L*dist)^2 || 
                @warn("(τ*δ)^2 + 2τ*ε*L ≤ (1-τ)*(L*dist)^2 fails")
        end
        =#

        #=
        if verbose
            mod(k, 20)==1 &&
            @printf("%4s %8s %8s %10s %10s %10s %6s\n", 
                "k", "ε", "δ", "||X - Y||", "f(X)", "Q(X, Y)", "t")
            @printf("%4d %8.1e %8.1e %10.1e %10.2e %10.2e %6.2f\n", 
                k, ε, δ, dist, fnew, Qnew, t)
        end
        =#

        # Update Y, Xold, t
        if accelerate
            tnew = (1 + √(1 + 4t^2))/2
        else
            tnew = 1.0
        end
        
        if method==:IR
            Y.data .= Xnew .- (t/tnew*τ/L).*V .+ ((t - 1)/tnew).*(Xnew .- Xold)
        else
            Y.data .= Xnew .+ ((t - 1)/tnew).*(Xnew .- Xold)
        end
        Xold .= Xnew
        t = tnew
    end
    
    return Xnew, y, fvals
end

n = 6
U, H = randncm(n)
ncm(U, H, kmax=20);

In [ ]:
n = 6
U, H = randncm(n)
kmax = 50
verbose = false

@time X0, y0, fvals0 = ncm(U, H, accelerate=false,  kmax=kmax, verbose=verbose)
@time X,  y,  fvals  = ncm(U, H, accelerate=true,   kmax=kmax, verbose=verbose)
@time X1, y1, fvals1 = ncm(U, H, method=:IR, τ=0.1, kmax=kmax, verbose=verbose)
@time X2, y2, fvals2 = ncm(U, H, method=:IR, τ=0.5, kmax=kmax, verbose=verbose)
@time X3, y3, fvals3 = ncm(U, H, method=:IR, τ=0.9, kmax=kmax, verbose=verbose)

plot(yaxis=:log, legend=:topright, size=(900,600), title="subproblem solver: JuMP with CSDP")
plot!(0:kmax, fvals0, label="PG", linestyle=:auto)
plot!(0:kmax, fvals, label="APG", linestyle=:auto)
plot!(0:kmax, fvals1, label=L"\tau = 0.1", markershape=:auto)
plot!(0:kmax, fvals2, label=L"\tau = 0.5", markershape=:auto)
plot!(0:kmax, fvals3, label=L"\tau = 0.9", markershape=:auto)
xlabel!(L"k"); ylabel!(L"f(x_k)")

In [ ]:
savefig("plot.pdf")

---

# ProjPSD

In [ ]:
struct ProjPSD
    nmax::Int
    jobz::Char
    range::Char
    il::Int
    iu::Int
    abstol::Float64
    m::Base.RefValue{Int}
    w::Vector{Float64}
    Z::Matrix{Float64}
    ldz::Int
    isuppz::Vector{Int}
    work::Vector{Float64}
    lwork::Int
    iwork::Vector{Int}
    liwork::Int
    info::Base.RefValue{Int}

    function ProjPSD(nmax)
        n = nmax
        A = Symmetric(zeros(1,1))

        jobz = 'V'
        range = 'V'
        lda = n
        vl = 0.0
        vu = Inf
        il = 0
        iu = 0
        abstol = -1.0
        m = Ref{Int}(0)
        w = zeros(n)
        Z = zeros(n, n)
        ldz = n
        isuppz = zeros(Int, 2n)
        work   = zeros(1)
        iwork  = zeros(Int, 1)
        info   = Ref{Int}(0)

        # Perform an optimal workspace query
        ccall((:dsyevr_64_, "libopenblas64_"), Cvoid,
            (Ref{UInt8}, Ref{UInt8}, Ref{UInt8}, Ref{Int},
                Ptr{Float64}, Ref{Int}, Ref{Float64}, Ref{Float64},
                Ref{Int}, Ref{Int}, Ref{Float64}, Ptr{Int},
                Ptr{Float64}, Ptr{Float64}, Ref{Int}, Ptr{Int},
                Ptr{Float64}, Ref{Int}, Ptr{Int}, Ref{Int},
                Ptr{Int}),
            jobz, range, A.uplo, n,
            A.data, lda, vl, vu,
            il, iu, abstol, m,
            w, Z, ldz, isuppz,
            work, -1, iwork, -1,
            info)

        lwork = Int(real(work[1]))
        liwork = iwork[1]

        resize!(work, lwork)
        resize!(iwork, liwork)

        new(nmax, jobz, range, il, iu, abstol, m, w, Z, ldz,
            isuppz, work, lwork, iwork, liwork, info)
    end
end


function (obj::ProjPSD)(A::Symmetric)
    lda, n = size(A)

    @assert lda == n
    @assert n <= obj.nmax

    vl = 0.0
    vu = Inf  # Tests show that vu = Inf is faster than
              # vu = min(norminf, normfro)
    abstol = -1.0

    #=
    vl = 1e-8
    norminf = ccall((:dlansy_64_, "libopenblas64_"), Cdouble,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, Ptr{Float64}, Ref{Int},
        Ptr{Float64}), 'I', A.uplo, n, A.data, lda, obj.work)
    normfro = ccall((:dlansy_64_, "libopenblas64_"), Cdouble,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, Ptr{Float64}, Ref{Int},
        Ptr{Float64}), 'F', A.uplo, n, A.data, lda, obj.work)
    vu = min(norminf, normfro)
    if vu < vl
        vu = 2*vl
    end
    abstol = 1e-8
    =#

    ccall((:dsyevr_64_, "libopenblas64_"), Cvoid,
    (Ref{UInt8}, Ref{UInt8}, Ref{UInt8}, Ref{Int},
        Ptr{Float64}, Ref{Int}, Ref{Float64}, Ref{Float64},
        Ref{Int}, Ref{Int}, Ref{Float64}, Ptr{Int},
        Ptr{Float64}, Ptr{Float64}, Ref{Int}, Ptr{Int},
        Ptr{Float64}, Ref{Int}, Ptr{Int}, Ref{Int},
        Ptr{Int}),
    obj.jobz, obj.range, A.uplo, n,
    A.data, lda, vl, vu,
    obj.il, obj.iu, abstol, obj.m,
    obj.w, obj.Z, obj.ldz, obj.isuppz,
    obj.work, obj.lwork, obj.iwork, obj.liwork,
    obj.info)

    k = obj.m[]
    λ, V = obj.w, obj.Z
    ldv = obj.ldz

    # V = V*diagm(sqrt.(λ))
    for j = 1:k
        tmp = sqrt(λ[j])
        for i = 1:n
            V[i,j] *= tmp
        end
    end

    # A = V*V'
    ccall((:dsyrk_64_, "libopenblas64_"), Cvoid,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, Ref{Int}, Ref{Float64},
            Ptr{Float64}, Ref{Int}, Ref{Float64}, Ptr{Float64}, Ref{Int}),
        A.uplo, 'N', n, k, 1.0, V, ldv, 0.0, A.data, lda)

    return A
end

n = 10

A = Symmetric(randn(n,n))
B = copy(A)

@time myproj = ProjPSD(n);
@time myproj(B) # Compile myproj
@time myproj(A);

---
# Optim.jl

In [ ]:
function ncm2(U, H, myproj; 
        τ=1.0, 
        t0=1.0,
        f_calls_limit=1000, 
        kmax=300,
        tol=1e-2,
        g_tol=1e-2, 
        algo=GradientDescent(), 
        method=:PG, 
        accelerate=true, 
        verbose=true,
    )
        
    # Check for valid input
    issymmetric(U) || error("U must be symmetric")
    issymmetric(H) || error("H must be symmetric")
    size(U) == size(H) || error("U and H must be the same size")
    
    # Create the projection function
    n = size(U, 1)
    
    H2 = Symmetric(H.*H)
    
    # Loss function and gradient
    f(X) = norm(H.*(X .- U))
    #∇f(X) = Symmetric(H2.*(X .- U))
    
    # Lipschitz constant of ∇f
    L = norm(H2)
    
    # Memory allocation
    y = zeros(n)
    g = zeros(n)
    d = zeros(n)
    M = Symmetric(zeros(n,n))
    #Y = Symmetric(triu(zeros(n,n),1) + I)
    Y = copy(M)
    Xold = copy(M)
    Xnew = copy(M)
    Λ = copy(M)
    Γ = copy(M)
    V = copy(M)
    X = copy(M)
    ∇fY = copy(M)
    fvals = Float64[]
    
    # Evaluates dual objective function and its gradient
    function dualobj!(ff, gg, y)
        ∇fY.data .= H2.*(Y .- U)
        M.data .= Y .- (τ/L).*(∇fY .+ Diagonal(y))
        X .= M
        myproj(X)
        
        # Update Λ and Γ
        Λ.data .= (L/τ).*(X .- M)
        Γ.data .= Diagonal(y) .- Λ
        
        # Ensure that diag(Xnew).==1 exactly
        for i=1:n
            d[i] = 1/sqrt(X[i,i])
        end
        for j=1:n
            for i=1:n
                Xnew.data[i,j] = d[i]*d[j]*X[i,j]
            end
        end
        
         M.data .= H.*(Xnew .- U)
        push!(fvals, 0.5*dot(M,M))

        V.data .= ∇fY .+ (L/τ).*(Xnew .- Y) .+ Γ

        if gg != nothing
            for i=1:n
                gg[i] = 1 - X[i,i]
            end
        end
        
        if ff != nothing
            normW = norm(view(myproj.w, 1:myproj.m[]))
            return sum(y) + (L/2τ)*(normW^2 - dot(Y,Y))
        end
    end
    
    # The callback function
    function cb(os)
        #println("================ CALLBACK ================")
        #@show os[end].metadata["x"]
        
        # Compute ε and δ
        ε = max(0.0, dot(Xnew, Λ))
        δ = norm(V)
        dist = norm(M.data .= Xnew .- Y)
        
        stop_optimize = false
        if (τ*δ)^2 + 2τ*ε*L ≤ (1-τ)*(L*dist)^2
            #@show (τ*δ)^2 + 2τ*ε*L, (1-τ)*(L*dist)^2
            stop_optimize = true
        end
        #if length(fvals) >= f_calls_limit
        #    stop_optimize = true
        #end
        #println("==========================================")
        return stop_optimize
    end
    
    # Define subproblem
    prob = Optim.only_fg!(dualobj!)
    opts = Optim.Options(
            g_tol=g_tol, 
            store_trace=true, 
            extended_trace=true, 
            show_trace=false, 
            callback=cb)
    
    Rp = Rd = Inf
    k = 0
    t = t0
    while max(Rp, Rd) > tol && k < kmax && length(fvals) < f_calls_limit
        k = k + 1
        
        res = optimize(prob, y, algo, opts)
        
        #if length(fvals) < f_calls_limit && !Optim.g_converged(res)
        #    error("Failed to solve subproblem: gnorm ≰ g_tol")
        #end
        
        gnorm = Optim.g_norm_trace(res)[end]
        
        y .= res.minimizer
        
        # Compute ε and δ
        ε = dot(Xnew, Λ)
        δ = norm(V)
        dist = norm(M.data .= Xnew .- Y)
        
        for i=1:n
            d[i] = 1 - Xnew[i,i]
        end
        Rp = norm(d)/(1 + √n)
        Rd = norm(M.data .= H2.*(Xnew .- U) .- Diagonal(y) .- Λ)
        
        if verbose
            mod(k, 20)==1 &&
            @printf("%4s %8s %8s %10s %10s %10s %10s %10s %10s %10s %10s\n", 
                "k", "f_calls", "g_calls", "||g||", "g_tol", "f(X)", "Rp", "Rd", "<X,Λ>", "||V||", "||X-Y||")
            @printf("%4d %8d %8d %10.2e %10.2e %10.2e %10.2e %10.2e %10.2e %10.2e %10.2e\n", 
                k, res.f_calls, res.g_calls, gnorm, g_tol, fvals[end], Rp, Rd, ε, δ, dist)
        end
        
        #=
        if (τ*δ)^2 + 2τ*ε*L > (1-τ)*(L*dist)^2 
            println("WARNING: (τ*δ)^2 + 2τ*ε*L ≤ (1-τ)*(L*dist)^2 fails")
            @show (τ*δ)^2 + 2τ*ε*L, (1-τ)*(L*dist)^2
        end
        =#

        # Update Y, Xold, t
        tnew = (1 + √(1 + 4t^2))/2
        Y.data .= Xnew .- (t/tnew*τ/L).*V .+ ((t - 1)/tnew).*(Xnew .- Xold)
        Xold .= Xnew
        t = tnew
    end
    #@show length(fvals), fvals[end]
    
    return Xnew, y, fvals
end

n = 6
myproj = ProjPSD(n)
U, H = randncm(n)
@time X, y, fvals = ncm2(U, H, myproj, 
    algo=BFGS(linesearch=HagerZhang()), 
    g_tol=1e-4, 
    τ=1.0, 
    f_calls_limit=200);

In [ ]:
#InitialStatic()
#InitialPrevious()
#InitialQuadratic()
#InitialConstantChange(αmin = 1e-12, αmax = 1.0, α0 = 1.0, ρ = 0.25, snap2one = (0.75, Inf))
#InitialHagerZhang()

In [ ]:
n = 100
myproj = ProjPSD(n)
U, H = randncm(n)
X = copy(U)

f_calls_limit=1000
tol=1e-1

alphaguess=InitialStatic(alpha=1.0)

plt = plot(yaxis=:log, legend=:topright, size=(900,600), title="subproblem solver: Optim.jl")
for algo in [BFGS] #[BFGS, LBFGS, ConjugateGradient, GradientDescent]
    for ls in [BackTracking] #[BackTracking, StrongWolfe, HagerZhang, MoreThuente, Static]
        for g_tol in [0.0]
            for τ in 0.85:0.05:0.95
                label="\\tau=$τ"
                println(label)
                @time X, y, fvals = ncm2(U, H, myproj,
                    τ=τ, 
                    f_calls_limit=f_calls_limit, 
                    tol=tol,
                    g_tol=g_tol, 
                    algo=algo(linesearch=ls(), alphaguess=alphaguess), 
                    verbose=false,
                )
                plot!(1:length(fvals), fvals, label=label)
            end
        end
    end
end
xlabel!("function evaluations"); ylabel!("objective function value")

In [ ]:
savefig("plot2.pdf")

---
# LineSearches.jl

In [ ]:
n = 6
U, H = randncm(n)

H2 = Symmetric(H.*H)
    
# Loss function and gradient
f(X) = 0.5*norm(H.*(X .- U))^2
∇f(X) = Symmetric(H2.*(X .- U))

# Lipschitz constant of ∇f
L = norm(H2)

k = 0
τ = 1.0
α = 1.0
ff = 0.0
g = zeros(n)
d = similar(g)
y = zeros(n)
M = Symmetric(zeros(n,n))
gg = zeros(n)
Y = copy(M)

myproj = ProjPSD(n)

function dualobj!(ff, gg, y, M)
    copy!(M.data, Y .- (τ/L).*(∇f(Y) .+ Diagonal(y)))
    myproj(M)
    if gg != nothing
        copy!(gg, 1 .- diag(M))
    end
    if ff != nothing
        normW = norm(myproj.w[1:myproj.m[]])
        return sum(y) + (L/2τ)*(normW^2 - dot(Y,Y))
    end
end

In [ ]:
k += 1

dobj = dualobj!(ff, g, y, M)
d .= -g
@show norm(g)

ϕ(t) = dualobj!(ff, nothing, y.+t.*d, M)
function dϕ(t)
    dualobj!(nothing, gg, y.+t.*d, M)
    return dot(d, gg)
end
ϕdϕ(t) = ϕ(t), dϕ(t)

α0 = α
ϕ0, dϕ0 = ϕdϕ(0.0)
α, ϕα = StrongWolfe()(ϕ, dϕ, ϕdϕ, α0, ϕ0, dϕ0)

xticks=[0.0, α, α0, 2α0]
yticks=[ϕ0, ϕ(α), ϕ(α0)]

plt = plot(ϕ, 0.0, α+10.0, 
    c=:blue, title="Iter. $k", legend=false, xticks=xticks, yticks=yticks)
plot!(t -> ϕ0 + t*dϕ0, 0.0, α+5.0, c=:red, s=:dash)
plot!(t -> ϕ(α) + (t-α)*dϕ(α), α, α+5.0, c=:black, s=:dash)
scatter!([0.0], [ϕ0], c=:red, ms=3)
scatter!([α], [ϕ(α)], c=:black, ms=3)
display(plt)

y .+= α.*d;

---
# plusdiag!(M, y)

In [ ]:
function plusdiag!(M::AbstractArray{T,2}, y::Vector{T}) where T
    for i=eachindex(y)
        @inbounds M[i,i] += y[i]
    end
    return M
end

function slowplusdiag!(X,y)
    X.data .= X .+ Diagonal(y)
    return X
end

In [ ]:
using BenchmarkTools

n = 100
X = Symmetric(zeros(n,n))
y = ones(n)

slowplusdiag!(copy(X),y)
plusdiag!(copy(X),y)

@btime slowplusdiag!($(copy(X)),y);
@btime plusdiag!($(copy(X)),y);

---
# LBFGSB.jl

In [ ]:
# Replaces linesearch function evaluations with last function evaluation for cleaner plots
function cleanvals!(vals, linesearchcalls)
    fgcalls = sum(linesearchcalls)
    v = view(vals, length(vals)-fgcalls+1:length(vals))
    a = 1
    for numcalls in linesearchcalls
        b = a + numcalls - 1
        fill!(view(v, a:b), v[b])
        a = b + 1
    end
    return vals
end

In [ ]:
isdiagallpos(X) = all(@inbounds X[i,i] > 0 for i=1:size(X,1))

In [ ]:
using Printf, LBFGSB

const START = b"START"
const FG = b"FG"
const STOP = b"STOP"
const NEW_X = b"NEW_X"

function calllbfgsb!(func!, g, y, 
        H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V, 
        fvals, resvals, Rpvals, Rdvals, L, τ, α, σ,
        n, memlim, wa, iwa, nbd, lower, upper, task, csave, lsave, isave, dsave;
        method=:IAPG,
        maxfgcalls=100,
        gtol=1e-2,
        exact=false,
        iprint=-1,
        verbose=false,
    )

    nRef = Ref{Cint}(n)
    mRef = Ref{Cint}(memlim)

    iprint = Ref{Cint}(iprint)    # print output at every iteration

    # "f is a DOUBLE PRECISION variable.
    # If the routine setulb returns with task(1:2)= 'FG', then f must be
    # set by the user to contain the value of the function at the point x."
    # "g is a DOUBLE PRECISION array of length n.
    # If the routine setulb returns with taskb(1:2)= 'FG', then g must be
    # set by the user to contain the components of the gradient at the
    # point x."
    fRef = Ref{Cdouble}(0.0)
    
    # specify the tolerances in the stopping criteria
    factr = Ref{Cdouble}(0.0)
    pgtol = Ref{Cdouble}(0.0)

    fgcalls = 0
    linesearchcalls = [0]
    sizehint!(linesearchcalls, 8)
    
    StopBFGS = false
    successful = true

    # "We start the iteration by initializing task."
    copyto!(task, START)

    while !StopBFGS

        # This is the call to the L-BFGS-B code.
        setulb(nRef, mRef, y, lower, upper, nbd,
            fRef, g, factr, pgtol, wa, iwa, task,
            iprint, csave, lsave, isave, dsave)

        if view(task, 1:2) == FG
            if fgcalls >= maxfgcalls
                copyto!(task, STOP)
            else
                fRef[] = func!(g, y, 
                    H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V, 
                    fvals, resvals, Rpvals, Rdvals, L, τ)
                fgcalls += 1
                linesearchcalls[end] += 1
                    
                if verbose
                    if fgcalls == 1
                        @printf("\n%8s %10s %10s %10s", 
                            "fgcalls", "fRef[]", "norm(g)", "gtol")
                    end
                    @printf("\n%8d %10.2e %10.2e %10.2e", 
                        fgcalls, fRef[], norm(g), gtol)
                end
                
                if !exact
                    if method==:IAPG
                        condition = (norm(g) < gtol)
                    else
                        ε = max(0.0, dot(Xnew, Λ))
                        δ = norm(V)
                        dist = norm(M.data .= Xnew .- Y)
                        if method==:IR
                            condition = ((τ*δ)^2 + 2τ*ε*L ≤ L*((1-τ)*L - α*τ)*dist^2)
                            verbose && @printf(" %10.2e, %10.2e", (τ*δ)^2 + 2τ*ε*L, L*((1-τ)*L - α*τ)*dist^2)
                        else # method==:IER
                            M.data .+= α.*V
                            β = norm(M)
                            condition = (β^2 + 2α*ε ≤ (σ*dist)^2)
                            verbose && @printf(" %10.2e, %10.2e", β^2 + 2α*ε, (σ*dist)^2)
                        end
                    end

                    if condition
                        copyto!(task, STOP)
                    end
                end
            end
            
            if fgcalls == 1 && view(task, 1:4) != STOP
                push!(linesearchcalls, 0)
            end

        elseif view(task, 1:5) == NEW_X
            verbose && @printf(" (linesearch complete)")
            push!(linesearchcalls, 0)
            
        else
            StopBFGS = true
            if !exact && view(task, 1:4) != STOP
                @printf("\ntask = %s\n", String(copy(task)))
                successful = false
            end
        end
    end
    verbose && @printf("\n")
    
    return successful, linesearchcalls
end

In [ ]:
struct NCMstorage
    n::Int
    memlim::Int
    y::Vector{Float64}
    g::Vector{Float64}
    d::Vector{Float64}
    M::Symmetric{Float64,Array{Float64,2}}
    H2::Symmetric{Float64,Array{Float64,2}}
    Y::Symmetric{Float64,Array{Float64,2}}
    ∇fY::Symmetric{Float64,Array{Float64,2}}
    Λ::Symmetric{Float64,Array{Float64,2}}
    Γ::Symmetric{Float64,Array{Float64,2}}
    V::Symmetric{Float64,Array{Float64,2}}
    X::Symmetric{Float64,Array{Float64,2}}
    Xold::Symmetric{Float64,Array{Float64,2}}
    Xnew::Symmetric{Float64,Array{Float64,2}}
    wa::Vector{Float64}
    iwa::Vector{Int32}
    nbd::Vector{Int32}
    lower::Vector{Float64}
    upper::Vector{Float64}
    task::Vector{UInt8}
    csave::Vector{UInt8}
    lsave::Vector{Int32}
    isave::Vector{Int32}
    dsave::Vector{Float64}

    function NCMstorage(n, memlim)
        y = zeros(n)
        g = zeros(n)
        d = zeros(n)

        M = Symmetric(zeros(n,n))
        H2 = copy(M)
        Y = copy(M)
        ∇fY = copy(M)
        Λ = copy(M)
        Γ = copy(M)
        V = copy(M)
        X = copy(M)
        Xold = copy(M)
        Xnew = copy(M)

        nmax = n
        mmax = memlim

        wa = zeros(Cdouble, 2mmax*nmax + 5nmax + 11mmax*mmax + 8mmax)
        iwa = zeros(Cint, 3nmax)

        # provide nbd which defines the bounds on the variables:
        nbd = zeros(Cint, nmax)         # no bounds on the variables
        lower = zeros(Cdouble, nmax)    # the lower bounds
        upper = zeros(Cdouble, nmax)    # the upper bounds

        task  = fill(Cuchar(' '), 60)   # fortran's blank padding
        csave = fill(Cuchar(' '), 60)   # fortran's blank padding
        lsave = zeros(Cint, 4)
        isave = zeros(Cint, 44)
        dsave = zeros(Cdouble, 29)

        new(n, memlim, y, g, d, M, H2, Y, ∇fY, Λ, Γ, V, X, Xold, Xnew, 
            wa, iwa, nbd, lower, upper, task, csave, lsave, isave, dsave)
    end
end


function ncm(U::AbstractArray{T,2}, H::AbstractArray{T,2}, myproj::ProjPSD, storage::NCMstorage; 
        method=:IAPG, 
        τ=1.0,
        α=0.0,
        σ=1.0,
        tol=1e-2,
        exact=false,
        kmax=2000, 
        f_calls_limit=2000, 
        verbose=false,
        lbfgsbverbose=false,
        cleanvals=true,
    ) where T
    
    # Loss function and gradient
    #f(X) = 0.5*norm(H.*(X .- U))^2
    #∇f(X) = Symmetric(H2.*(X .- U))
    
    y = storage.y
    g = storage.g
    d = storage.d
    M = storage.M
    H2 = storage.H2
    Y = storage.Y
    ∇fY = storage.∇fY
    Λ = storage.Λ
    Γ = storage.Γ
    V = storage.V
    X = storage.X
    Xold = storage.Xold
    Xnew = storage.Xnew
    
    fill!(y, 0)
    fill!(g, 0)
    fill!(d, 0)
    fill!(M, 0)
    fill!(H2, 0)
    fill!(Y, 0)
    fill!(∇fY, 0)
    fill!(Λ, 0)
    fill!(Γ, 0)
    fill!(V, 0)
    fill!(X, 0)
    fill!(Xold, 0)
    fill!(Xnew, 0)
    
    memlim = storage.memlim
    wa = storage.wa
    iwa = storage.iwa
    nbd = storage.nbd
    lower = storage.lower
    upper = storage.upper
    task = storage.task
    csave = storage.csave
    lsave = storage.lsave
    isave = storage.isave
    dsave = storage.dsave
        
    # Check for valid input
    method in [:IAPG, :IR, :IER] || error("method must be :IAPG, :IR, or :IER")
    println("$method method")

    issymmetric(U) || error("U must be symmetric")
    issymmetric(H) || error("H must be symmetric")
    size(U)==size(H) || error("U and H must be the same size")
    
    n = size(U, 1)
    n==storage.n || error("n != storage.n")
        
    H2.data .= H.*H
    
    # Lipschitz constant of ∇f
    L = norm(H2)
    
    if method==:IAPG
        τ==1 || error("IAPG method requires τ = 1")
        t0 = 1.0
    end
    
    if method==:IR
        0 < τ ≤ 1 || error("IR method requires 0 < τ ≤ 1")
        0 ≤ α ≤ (1 - τ)*(L/τ) || error("IR method requires 0 ≤ α ≤ $((1 - τ)*(L/τ))")
        t0 = 1.0
    end
    
    if method==:IER
        τ==1 || error("IER method requires τ = 1")
        α > 1/L || error("IER method requires α > $(1/L)")
        0 ≤ σ ≤ 1 || error("IER method requires 0 ≤ σ ≤ 1")
        λ = α/(1 + α*L)
        t0 = 0.0
    end
    
    # Evaluates dual objective function and its gradient
    function dualobj!(gg, y, 
            H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V, 
            fvals, resvals, Rpvals, Rdvals, L, τ)

        ∇fY.data .= H2.*(Y .- U)
        M .= ∇fY; plusdiag!(M, y)  # M = ∇f(Y) + Diag(y)
        M.data .= Y .- (τ/L).*M    # M = Y - (τ/L)*(∇f(Y) + Diag(y))
        X .= M
        myproj(X)

        # Update Λ and Γ
        Λ.data .= (L/τ).*(X .- M)       # Λ is psd
        Γ.data .= .-Λ; plusdiag!(Γ, y)  # Γ = Diag(y) - Λ

        # Ensure that diag(Xnew).==1 exactly
        if isdiagallpos(X)
            for i=1:n
                @inbounds d[i] = 1/sqrt(X[i,i])
            end
            for j=1:n
                for i=1:n
                    @inbounds Xnew.data[i,j] = d[i]*d[j]*X[i,j]
                end
            end
        end

        # Update V
        V.data .= ∇fY .+ (L/τ).*(Xnew .- Y) .+ Γ

        # Compute and store the optim. cond. residual
        for i=1:n
            @inbounds d[i] = 1 - Xnew[i,i]
        end
        Rp = norm(d)/(1 + √n)
        Rd = norm(M.data .= H2.*(Xnew .- U) .+ Γ)
        push!(Rpvals, Rp)
        push!(Rdvals, Rd)
        push!(resvals, max(Rp,Rd))

        # Compute and store the objective function
        M.data .= H.*(Xnew .- U)
        push!(fvals, 0.5*dot(M,M))

        # Compute the gradient of the dual function
        for i=1:n
            @inbounds gg[i] = 1 - X[i,i]
        end

        w = view(myproj.w, 1:myproj.m[])
        return sum(y) + (L/2τ)*(dot(w,w) - dot(Y,Y))
    end
    
    k = 0
    t = t0
    gtol = NaN
    Rp = Rd = Inf
    innersuccess = true
    fvals   = Float64[]
    resvals = Float64[]
    Rpvals  = Float64[]
    Rdvals  = Float64[]
    
    while ( #innersuccess && 
            max(Rp, Rd) > tol && 
            k < kmax && 
            length(fvals) < f_calls_limit )
        
        k += 1
        
        if method==:IER
            tnew = t + (λ + √(λ^2 + 4λ*t))/2
            Y.data .= (t/tnew).*Xnew .+ ((tnew - t)/tnew).*Xold
        else
            tnew = (1 + √(1 + 4t^2))/2
        end
        
        # Reset L-BFGS-B arrays
        fill!(wa, 0.0)
        fill!(iwa, 0)
        fill!(task, Cuchar(' '))
        fill!(csave, Cuchar(' '))
        fill!(lsave, 0)
        fill!(isave, 0)
        fill!(dsave, 0.0)
            
        maxfgcalls = f_calls_limit - length(fvals)

        if method==:IAPG
            gtol = (1 + √n)*min(1/tnew^3.1, 0.2*Rd)
        end
        
        if exact
            gtol = 0.0
        end
        
        # Solve the subproblem
        innersuccess, linesearchcalls = calllbfgsb!(dualobj!, g, y, 
            H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V, fvals, resvals, Rpvals, Rdvals, L, τ, α, σ,
            n, memlim, wa, iwa, nbd, lower, upper, task, csave, lsave, isave, dsave,
            method=method,
            maxfgcalls=maxfgcalls,
            gtol=gtol,
            exact=exact,
            verbose=lbfgsbverbose,
        )
        innersuccess || println("Failed to solve subproblem.")
        fgcalls = sum(linesearchcalls)

        if cleanvals
            cleanvals!(fvals, linesearchcalls)
            cleanvals!(resvals, linesearchcalls)
        end

        Rp = Rpvals[end]
        Rd = Rdvals[end]
        ε = dot(Xnew, Λ)
        δ = norm(V)
        dist = norm(M.data .= Xnew .- Y)

        if verbose
            mod(k, 20)==1 &&
            @printf("%4s %8s %10s %10s %14s %10s %10s %10s %10s %10s\n", 
                "k", "fgcalls", "||g||", "gtol", "f(X)", "Rp", "Rd", "<X,Λ>", "||V||", "||X-Y||")
            @printf("%4d %8d %10.2e %10.2e %14.6e %10.2e %10.2e %10.2e %10.2e %10.2e\n", 
                k, fgcalls, norm(g), gtol, fvals[end], Rp, Rd, ε, δ, dist)
        end
        
        if !exact
            if method==:IR
                ε = max(0.0, ε)
                condition = ((τ*δ)^2 + 2τ*ε*L ≤ L*((1-τ)*L - α*τ)*dist^2)
                if !condition && (length(fvals) < f_calls_limit)
                    println("WARNING: (τ*δ)^2 + 2τ*ε*L ≤ L*((1-τ)*L - α*τ)*dist^2 fails")
                    #@show (τ*δ)^2 + 2τ*ε*L, L*((1-τ)*L - α*τ)*dist^2
                end
            end

            if method==:IER
                ε = max(0.0, ε)
                M.data .+= α.*V
                β = norm(M)
                condition = (β^2 + 2α*ε ≤ (σ*dist)^2)
                if !condition && (length(fvals) < f_calls_limit)
                    println("WARNING: β^2 + 2α*ε ≤ (σ*dist)^2 fails")
                    #@show β^2 + 2α*ε, (σ*dist)^2
                end
            end
        end

        # Update
        if method==:IAPG
            Y.data .= Xnew .+ ((t - 1)/tnew).*(Xnew .- Xold)
            Xold .= Xnew
        end
        
        if method==:IR
            Y.data .= Xnew .- ((t/tnew)*(τ/L)).*V .+ ((t - 1)/tnew).*(Xnew .- Xold)
            Xold .= Xnew
        end
        
        if method==:IER
            Xold.data .-= (tnew - t).*(V .+ L.*(Y .- Xnew))
        end
        
        t = tnew
    end
    
    if max(Rp, Rd) > tol
        println("Failed to converge after $(length(fvals)) function evaluations.")
    else
        println("Converged after $(length(fvals)) function evaluations.")
    end
    
    return Xnew, y, fvals, resvals
end

In [ ]:
n, γ = 100, 0.1
memlim = 10

verbose=true

U, H = randncm(n, γ=γ)

myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

@time ncm(U, H, myproj, storage, verbose=verbose);
@time ncm(U, H, myproj, storage, exact=true, verbose=verbose);

In [ ]:
n, γ = 100, 0.1
memlim = 10

U, H = randncm(n, γ=γ)

myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

plot(yaxis=:log, size=(900,600), title="n=$n, \\gamma=$γ", 
    xlabel="function evaluations", ylabel=L"\max\{R_p,R_d\}")
#@time X, y, fvals, resvals = ncm(U, H, myproj, storage, exact=true);
#plot!(1:length(resvals), resvals, label="APG")
#@time X, y, fvals, resvals = ncm(U, H, myproj, storage);
#plot!(1:length(resvals), resvals, label="IAPG")
#@time X, y, fvals, resvals = ncm(U, H, myproj, storage, method=:IR, τ=0.95)
#plot!(1:length(resvals), resvals, label="IR, \\tau=0.95, \\alpha=0")
#@time X, y, fvals, resvals = ncm(U, H, myproj, storage, method=:IR, τ=0.95, α=0.25)
#plot!(1:length(resvals), resvals, label="IR, \\tau=0.95, \\alpha=0.25")
@time X, y, fvals, resvals = ncm(U, H, myproj, storage, method=:IER, α=0.25, exact=true)
plot!(1:length(resvals), resvals, label="EER, \\alpha=0.25, \\sigma=1.0")
@time X, y, fvals, resvals = ncm(U, H, myproj, storage, method=:IER, α=0.25)
plot!(1:length(resvals), resvals, label="IER, \\alpha=0.25, \\sigma=1.0")

In [ ]:
savefig("plot4.pdf")

In [ ]:
n, γ = 100, 1.0
memlim = 10

U, H = randncm(n, γ=γ)

myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

verbose=false

@time X, y, fvals, resvals = ncm(U, H, myproj, storage, method=:IAPG, verbose=verbose)
@time X, y, fvals, resvals = ncm(U, H, myproj, storage, method=:IR, τ=0.95, verbose=verbose);

In [ ]:
n, γ = 587, 0.05
memlim = 10
U, H = randncm(n, γ=γ, seed=0)
myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

f_calls_limit=kmax=8000
verbose=false
tol=1e-1

plt = plot(yaxis=:log, legend=:topright, size=(900,600), title="n=$n, \\gamma=$γ",
    xlabel="function evaluations", ylabel=L"\max\{R_p,R_d\}")

@time X, y, fvals, resvals = ncm(U, H, myproj, storage, 
    method=:IAPG, 
    f_calls_limit=f_calls_limit, 
    kmax=kmax,
    tol=tol,
    verbose=verbose,
)
plot!(1:length(resvals), resvals, label="IAPG")

for τ in [0.95]
    @time X, y, fvals, resvals = ncm(U, H, myproj, storage, 
        method=:IR,
        τ=τ, 
        f_calls_limit=f_calls_limit,
        kmax=kmax,
        tol=tol,
        verbose=verbose,
    )
    plot!(1:length(resvals), resvals, label="IR, \\tau=$τ")
end
plt

In [ ]:
savefig("plot3.pdf")

---
# Comparing `ncm`, `ncm2`, and `ncm3`

In [ ]:
n, γ = 6, 1.0
myproj = ProjPSD(n)
U, H = randncm(n, γ=γ)
H.*U

In [ ]:
@time X0, y0, fvals0 = ncm0(U, H)
0.5*norm(H.*(X0 .- U))^2

In [ ]:
@time X1, y1, fvals1 = ncm(U, H, verbose=false)
0.5*norm(H.*(X1 .- U))^2

In [ ]:
@time X2, y2, fvals2 = ncm2(U, H, myproj, tol=1e-4, verbose=false)
0.5*norm(H.*(X2 .- U))^2

In [ ]:
@time X3, y3, fvals3 = ncm3(U, H, myproj, tol=1e-4, verbose=false)
0.5*norm(H.*(X3 .- U))^2

In [ ]:
[y0 y1 y2 y3]